In [ ]:
from __future__ import division, print_function
import sys
import os
import glob
import numpy as np


from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model
from keras.preprocessing import image


from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer


app = Flask(__name__)


MODEL_PATH = 'C:/Users/shubh/Downloads/keras-flask-deploy-webapp-master/models/breast_cancer_model.h5'


model = load_model(MODEL_PATH)
model._make_predict_function()          

print('Model loaded. Check http://127.0.0.1:5000/')


def model_predict(img_path, model):
    img = image.load_img(img_path, target_size=(64, 64))

   
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

   
    x = preprocess_input(x, mode='caffe')

    preds = model.predict(x)
    return preds


@app.route('/', methods=['GET'])
def index():
    
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        
        f = request.files['image']

        
        basepath = os.path.dirname('__file__')
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)

        
        preds = model_predict(file_path, model).tolist()
        s=''
        for i in  preds:
            s+=str(i)
            return s
    return None


if __name__ == '__main__':
    
    http_server = WSGIServer(('0.0.0.0', 5000), app)
    http_server.serve_forever()


Using TensorFlow backend.


Model loaded. Check http://127.0.0.1:5000/


127.0.0.1 - - [2019-12-16 18:54:36] "GET / HTTP/1.1" 200 1957 0.062664
127.0.0.1 - - [2019-12-16 18:54:39] "GET /favicon.ico HTTP/1.1" 404 341 0.000000
127.0.0.1 - - [2019-12-16 18:55:11] "POST /predict HTTP/1.1" 200 120 0.347890
